<a href="https://colab.research.google.com/github/plthiyagu/Personnel/blob/master/Apache_Spark_3_0_ML_on_GPU_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Aug 28 22:43:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install findspark

In [3]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
!wget https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.0.0-0.1.0/xgboost4j-spark_3.0-1.0.0-0.1.0.jar
!wget http://insecure.repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/0.1.0/rapids-4-spark_2.12-0.1.0.jar

--2020-08-28 22:45:01--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.14/cudf-0.14-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.36.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.36.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145993287 (139M) [application/java-archive]
Saving to: ‘cudf-0.14-cuda10-1.jar’

cudf-0.14-cuda10-1. 100%[===================>] 139.23M   209MB/s    in 0.7s    

2020-08-28 22:45:02 (209 MB/s) - ‘cudf-0.14-cuda10-1.jar’ saved [145993287/145993287]

--2020-08-28 22:45:02--  https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.0.0-0.1.0/xgboost4j_3.0-1.0.0-0.1.0.jar
Resolving repo1.maven.org (repo1.maven.org)... 151.101.36.209
Connecting to repo1.maven.org (repo1.maven.org)|151.101.36.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231556205 (221M) [application/java-archive]
Saving to: ‘xgboost4j_3.0-1.0.0-0.1.0.jar’

xgboost4j_3.0-1.0.0 100%[==================

In [4]:
!ls

CatBoostdata.csv	       sample_data
covtype_test.parquet	       spark-3.0.0-bin-hadoop2.7
covtype_train.parquet	       spark-3.0.0-bin-hadoop2.7.tgz
cudf-0.14-cuda10-1.jar	       xgboost4j_3.0-1.0.0-0.1.0.jar
rapids-4-spark_2.12-0.1.0.jar  xgboost4j-spark_3.0-1.0.0-0.1.0.jar


In [5]:
!pwd

/content


In [6]:
!ls /content/rapids-4-spark_2.12-0.1.0.jar

/content/rapids-4-spark_2.12-0.1.0.jar


In [7]:
import os

In [8]:
!ls /usr/lib/jvm/

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [9]:
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"]="/content/spark-3.0.0-bin-hadoop2.7"

In [10]:
os.environ["PYSPARK_SUBMIT_ARGS"]='--jars /content/cudf-0.14-cuda10-1.jar,/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar,/content/cudf-0.14-cuda10-1.jar,/content/rapids-4-spark_2.12-0.1.0.jar,/content/spark-3.0.0-bin-hadoop2.7.tgz pyspark-shell'

In [11]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").config("spark.plugins","com.nvidia.spark.SQLPlugin").config("spark.rapid.memory.gpu.pooling.enabled",False).getOrCreate()
spark.sparkContext.addPyFile('/content/xgboost4j-spark_3.0-1.0.0-0.1.0.jar')
spark.sparkContext.addPyFile('/content/rapids-4-spark_2.12-0.1.0.jar')

In [12]:
reader= spark.read

In [13]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel,XGBoostClassifier

import pandas as pd
import numpy as np

In [ ]:
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/covtype_train.parquet
!wget https://raw.githubusercontent.com/plthiyagu/Personnel/master/Dataset/covtype_test.parquet

In [14]:
!ls -lart

total 600596
drwxr-xr-x 13 1000 1000      4096 Jun  6 12:09 spark-3.0.0-bin-hadoop2.7
-rw-r--r--  1 root root 220272364 Jun  6 13:35 spark-3.0.0-bin-hadoop2.7.tgz
-rw-r--r--  1 root root 145993287 Jun  9 13:17 cudf-0.14-cuda10-1.jar
-rw-r--r--  1 root root   6676958 Jun 20 02:24 rapids-4-spark_2.12-0.1.0.jar
-rw-r--r--  1 root root 231556205 Jun 26 03:20 xgboost4j_3.0-1.0.0-0.1.0.jar
-rw-r--r--  1 root root   2040779 Jun 26 03:21 xgboost4j-spark_3.0-1.0.0-0.1.0.jar
drwxr-xr-x  1 root root      4096 Aug 27 16:39 sample_data
drwxr-xr-x  1 root root      4096 Aug 27 16:39 .config
drwxr-xr-x  1 root root      4096 Aug 28 22:42 ..
-rw-r--r--  1 root root     14282 Aug 28 22:43 CatBoostdata.csv
-rw-r--r--  1 root root   1591040 Aug 28 22:43 covtype_test.parquet
-rw-r--r--  1 root root   6827427 Aug 28 22:43 covtype_train.parquet
drwxr-xr-x  1 root root      4096 Aug 28 22:45 .


In [15]:
import pyarrow.parquet as pq

In [16]:
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [17]:
train_data= reader.format('parquet').load('covtype_train.parquet')
test_data= reader.format('parquet').load('covtype_test.parquet')

In [18]:
train_data.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wilderness_Area1|Wilderness_Area2|Wilderness_Area3|Wilder

In [19]:
train_data.schema

StructType(List(StructField(Elevation,DoubleType,true),StructField(Aspect,DoubleType,true),StructField(Slope,DoubleType,true),StructField(Horizontal_Distance_To_Hydrology,DoubleType,true),StructField(Vertical_Distance_To_Hydrology,DoubleType,true),StructField(Horizontal_Distance_To_Roadways,DoubleType,true),StructField(Hillshade_9am,DoubleType,true),StructField(Hillshade_Noon,DoubleType,true),StructField(Hillshade_3pm,DoubleType,true),StructField(Horizontal_Distance_To_Fire_Points,DoubleType,true),StructField(Wilderness_Area1,DoubleType,true),StructField(Wilderness_Area2,DoubleType,true),StructField(Wilderness_Area3,DoubleType,true),StructField(Wilderness_Area4,DoubleType,true),StructField(Soil_Type1,DoubleType,true),StructField(Soil_Type2,DoubleType,true),StructField(Soil_Type3,DoubleType,true),StructField(Soil_Type4,DoubleType,true),StructField(Soil_Type5,DoubleType,true),StructField(Soil_Type6,DoubleType,true),StructField(Soil_Type7,DoubleType,true),StructField(Soil_Type8,DoubleType

In [20]:
pq_file=pq.read_table('covtype_train.parquet')

In [22]:
pq_file.shape

(464810, 55)

In [21]:
pq_file.columns

[<Column name='Elevation' type=DataType(double)>
 [
   [
     3381,
     3039,
     3318,
     2823,
     3211,
     2797,
     3385,
     3065,
     3209,
     3004,
     ...
     3059,
     3009,
     2179,
     3139,
     3064,
     3198,
     2975,
     3022,
     3133,
     2869
   ]
 ], <Column name='Aspect' type=DataType(double)>
 [
   [
     0,
     291,
     353,
     67,
     16,
     129,
     282,
     0,
     157,
     151,
     ...
     341,
     60,
     108,
     277,
     59,
     252,
     253,
     97,
     147,
     168
   ]
 ], <Column name='Slope' type=DataType(double)>
 [
   [
     18,
     12,
     21,
     13,
     19,
     9,
     8,
     4,
     7,
     18,
     ...
     22,
     24,
     31,
     17,
     26,
     3,
     15,
     9,
     7,
     10
   ]
 ], <Column name='Horizontal_Distance_To_Hydrology' type=DataType(double)>
 [
   [
     497,
     361,
     309,
     170,
     350,
     0,
     899,
     242,
     309,
     90,
     ...
     700,
     153

In [23]:
label="target"
features=[x for x in pq_file.column_names if x != label ]

In [24]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [31]:
import time
params={
    'eta':0.1,
    'gamma':0.1,
    'missing':0.0,
    'treeMethod':'gpu_hist',
    'maxDepth':6,
    'growPolicy':'depthwise',
    'lambda_':1.0,
    'subsample':1.0,
    'numRound':1000,
    'numWorkers':1,
    'verbosity':2
}

classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/content/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.0.0-bin-hadoop2.7/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1212, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving


Py4JError: ignored

In [30]:
import time
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time:  %s seconds" % (str(time.time()-start_time)))

NameError: ignored

In [ ]:
!nvidia-smi

Fri Aug 28 18:57:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
model.write().overwrite().save('/content/model/')

In [ ]:
!ls

In [ ]:
loaded_model= XGBoostClassificationModel().load('/content/model')

In [ ]:
result = loaded_model.transform(test_data)

In [ ]:
result.select(label,'rawPrediction','probability','prediction').show(5)